In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

In [13]:
%%time

#counter to check the number of house ads scraped
house_no =0

#dicitionary to store housing details
house_details = {}

#searching criteria
regex = re.compile('[ft2]+')
furn_search = re.compile('furnished')
type_search = re.compile('apartment|condo|duplex|flat|house|townhouse|land')
parking_search = re.compile('attached garage|detached garage|street parking|no parking')

#original url
url = 'https://vancouver.craigslist.org/d/real-estate/search/rea'

#looping through each main page
while True:
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, 'html.parser')
    houses = soup.find_all('p', class_='result-info')
    #looping through each ad in a page
    for house in houses:
        link = house.find('a', class_='result-title hdrlnk').get('href')
        #print(link)
        house_response = requests.get(link)
        house_data = house_response.text
        house_soup = BeautifulSoup(house_data, 'html.parser')
        
        #Getting the price
        price_tag = house_soup.find('span', class_='price')
        price = price_tag.text[1:] if price_tag else 'NA'
        
        #Getting area and bedroom
        housing_tag = house_soup.find('span', class_='housing')

        if housing_tag:
            bedroom = housing_tag.text[2:3]
            #print(len(housing_tag.text))
            #print(housing_tag.text)

            c = housing_tag.text.split('-')
            for i in c:
                i = i.strip()
                #print(i)
                ft = re.compile('[ft2]+')
                start_post = re.compile('\s')
                if ft.search(i):
                        #print(i)
                        end= regex.search(i).span()[0]
                        #print(regex.search(i).span())
                        #print(end)
                        if start_post.search(i):
                            start = start_post.search(i).span()[1]
                            #print(start_post.search(i).span())
                        else:
                            start =0
                        #print(start)
                        #print(i[start:end])
                        area = i[start:end]

            if area is None:
                area = 'NA'
        else:
            area = 'NA'
            bedroom ='NA'

        #Getting title of the ad
        title_tag = house_soup.find('span', id='titletextonly')
        title = title_tag.text if title_tag else 'NA'
        #print(title)
        
        #Getting the neighborhood
        place_tag = house_soup.find('small')
        place = place_tag.text[2:-1] if place_tag else 'NA'

        #Getting the posted date time
        post_datetime_tag = house_soup.find('time', class_='date timeago')
        post_datetime = post_datetime_tag['datetime'] if post_datetime_tag else 'NA'

        divs = house_soup.find('div',class_='mapAndAttrs')
        #print(divs.text)
        #checking if the estate is furnished or unfurnished

        if (furn_search.search(divs.text) is not None):
            finishing = 'furnished'
        else:
            finishing = 'unfurnished'

        #checking the housing type
        if (type_search.search(divs.text) is not None):
            house_type = type_search.search(divs.text).group()
        else:
            house_type = 'NA'

        #checking the parking type
        if (parking_search.search(divs.text) is not None):
            parking_type = parking_search.search(divs.text).group()
        else:
            parking_type = 'NA'

        #checking number of bedroom and bathroom
        data = divs.findNext('p').findNext('span')
        #print(data)
        if ((data.text is not None) and (data.text.find('/')>0)):
            # if there is no value in the bedroom, then check this value
            if bedroom is None:
                bd = data.text.split('/')[0].strip() 
                #print(bd)
                if (bd.find('BR')>0):
                    bedroom = bd[:-2]
                else:
                    bedroom = 'NA'
            #setting the number of bathroom value
            if(data.text.split('/')[1].strip()):
                bth = data.text.split('/')[1].strip()
                #print(bth)
                if (bth.find('Ba')>0):
                    bathroom = bth[:-2]
                else:
                    bathroom = 'NA'

        house_no+=1
    
        #Making an entry for each ad in a dictionary
        house_details[house_no] = [price, bedroom, area, title, place, post_datetime, finishing, house_type, parking_type,
                                   bathroom, link]
     
    #Getting the next page link in the original link
    url_tag = soup.find('a', class_='button next')
    if url_tag.get('href'):
        url = 'https://vancouver.craigslist.org/d/real-estate/search/rea' + url_tag.get('href')
        #print(url)
    else:
        # breaking from the loop in case there is no more link .ie in the last page
        break
    
    #putting a counter to fetch only 1200
    if(house_no==4000):
        break

#Building a data frame from the dictionary created, updated in the above loop
house_df = pd.DataFrame.from_dict(house_details,orient='index', columns=['Price', 'Bedroom', 'Area', 'Title', 'Place', 'Post Timing',
                                                                        'Finishing','House Type', 'Parking Type','Bathroom','Link'])

Wall time: 25min 51s


In [14]:
house_df

,Price,Bedroom,Area,Title,Place,Post Timing,Finishing,House Type,Parking Type,Bathroom,Link
1,NA,NA,NA,Semiahmoo Town Centre 7500 sq ft lot,NA,2020-07-20T19:05:26-0700,unfurnished,apartment,NA,1,https://vancouver.craigslist.org/rds/reo/d/bla...
2,1324888,7,7771,** OPEN HOUSE JULY 26 FOR THE BEST PRICED HOME...,Newton,2020-07-25T15:47:15-0700,unfurnished,house,NA,1,https://vancouver.craigslist.org/rds/reb/d/sur...
3,728000,1,547,***INVESTOR ALERT - AIRBNB ALLOWED*** Downtow...,Whistler,2020-07-03T22:11:07-0700,unfurnished,apartment,NA,1,https://vancouver.craigslist.org/van/reb/d/daw...
4,1098800,6,547,6 bed house with a basement suite in Sullivan ...,Surrey,2020-07-25T14:37:59-0700,unfurnished,apartment,NA,4.5,https://vancouver.craigslist.org/rds/reb/d/sur...
5,632000,2,1305,High Level Executive 2 Bedroom Condo in Coquitlam,North Coquitlam,2020-07-25T14:42:39-0700,unfurnished,condo,attached garage,2,https://vancouver.craigslist.org/pml/reb/d/coq...
...,...,...,...,...,...,...,...,...,...,...,...
2784,728000,2,1174,Fantastic 2 Bed / 3 Bath Townhouse in Coquitlam!,6 638 REGAN AVENUE,2020-05-13T04:03:57-0700,unfurnished,townhouse,NA,3,https://vancouver.craigslist.org/pml/reb/d/coq...
2785,958000,2,90,Cool 2 Bed / 2 Bath Condo in Vancouver!,1007 480 ROBSON STREET,2020-05-13T03:47:06-0700,unfurnished,condo,NA,2,https://vancouver.craigslist.org/van/reb/d/van...
2786,799900,3,1163,Spacious 3 Bed / 3 Bath Townhouse in Burnaby!,115 7499 6TH STREET,2020-05-27T00:10:34-0700,unfurnished,townhouse,NA,3,https://vancouver.craigslist.org/bnc/reb/d/bur...
2787,484000,1,60,Amazing 1 Bed / 1 Bath Condo in Coquitlam!,311 700 CLARKE ROAD,2020-05-26T23:12:59-0700,unfurnished,condo,NA,1,https://vancouver.craigslist.org/pml/reb/d/coq...


In [15]:
#Saving the dataframe into csv file
house_df.to_csv("Craiglist_HouseData3.csv")

In [18]:
house_df.shape

(2788, 11)